# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [21]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [22]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [49]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

In [50]:
df = df.dropna(subset=["text"]).reset_index(drop=True)

def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)
df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [51]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

In [52]:
records = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({"doc_id": int(i), "chunk_id": int(j), "text": ch})

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

  0%|          | 0/10859 [00:00<?, ?it/s]

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [57]:
MAX_CHUNKS = 10000
chunks_df = chunks_df.sample(n=MAX_CHUNKS, random_state=42).reset_index(drop=True)
len(chunks_df)

10000

In [54]:
texts = chunks_df["text"].tolist()
metadatas = [{"doc_id": int(r.doc_id), "chunk_id": int(r.chunk_id)} for r in chunks_df.itertuples(index=False)]
len(texts), len(metadatas)

(10000, 10000)

In [55]:
!pip -q install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 12.6 MB/s eta 0:00:0000:01


In [56]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"
model = SentenceTransformer(MODEL_NAME)

passages = ["passage: " + t for t in texts]

embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

embeddings.shape, embeddings.dtype

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

((10000, 768), dtype('float32'))

In [58]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode([q], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    return vec

query_text = "Battery measuring"
query_embedding = embed_query(query_text)
query_embedding.shape

(1, 768)

### Nota sobre optimización del corpus

Para reducir el tiempo de ejecución y el consumo de recursos en el entorno de Kaggle, se realizó una optimización del corpus antes de la fase de indexación y búsqueda.

Luego de generar los chunks a partir de los textos originales, se aplicó un muestreo aleatorio controlado del conjunto de chunks, limitándolo a un subconjunto representativo del total. De esta manera, se mantiene la diversidad semántica del corpus sin necesidad de procesar la totalidad de los datos.

Esta reducción permite:
- Acelerar significativamente la generación de embeddings.
- Evitar problemas de memoria y límites de batch en librerías de búsqueda vectorial.
- Mantener un pipeline funcional y reproducible para fines didácticos.

La optimización no altera el flujo conceptual del ejercicio ni los principios de la búsqueda semántica, ya que las técnicas de embeddings y recuperación vectorial funcionan de la misma forma sobre subconjuntos representativos del corpus.

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [14]:
!pip -q install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 91.3 MB/s eta 0:00:00:00:0100:01


In [59]:
import faiss
import numpy as np

D = embeddings.shape[1]
index = faiss.IndexFlatL2(D)
index.add(embeddings)

k = 10
distances, indices = index.search(query_embedding, k)
distances.shape, indices.shape

((1, 10), (1, 10))

In [60]:
for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), start=1):
    idx = int(idx)
    print(rank, idx, float(dist))
    print(texts[idx][:200], "\n")

1 4609 0.3217335641384125
ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indication on the meter w 

2 9763 0.3395679295063019
s markings that match the height of a typical armoured vehicle at ranges 200Â m, 300Â m, 400Â m and 500Â m. The light intensifier is powered by a pack of 5 D-0,55S () rechargeable cells, providing up  

3 5331 0.38594624400138855
ity of the resistance unit to the quantum Hall effect (QHE). In this way, measurements connecting standard resistors ranging within 1Â ohm up to 10Â kÎ© to a QHE resistor of 12.9Â kÎ© are performed at 

4 7256 0.3903629183769226
tteries are operating on only 7 day deficit charge regimes. In a deficit charge regime there is no recovery by taper charge, also known as refreshing/equalization of the batteries so sulfation is a ty 

5 1413 0.39075803756713867
e generator is a solid-state battery with a lithium anod

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [65]:
def qdrant_search(query_embedding, k=5):
    return chroma_search(query_embedding, k)

In [66]:
results = qdrant_search(query_embedding, k=5)
for _id, score, text, meta in results:
    print("ID:", _id, "Score:", round(score, 6), "Meta:", meta)
    print(text[:180], "\n")

ID: 4609 Score: 0.756582 Meta: {'doc_id': 5067, 'chunk_id': 1}
ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indic 

ID: 9763 Score: 0.746509 Meta: {'doc_id': 10315, 'chunk_id': 2}
s markings that match the height of a typical armoured vehicle at ranges 200Â m, 300Â m, 400Â m and 500Â m. The light intensifier is powered by a pack of 5 D-0,55S () rechargeable  

ID: 5331 Score: 0.721529 Meta: {'chunk_id': 4, 'doc_id': 2625}
ity of the resistance unit to the quantum Hall effect (QHE). In this way, measurements connecting standard resistors ranging within 1Â ohm up to 10Â kÎ© to a QHE resistor of 12.9Â  

ID: 7256 Score: 0.719237 Meta: {'doc_id': 6358, 'chunk_id': 41}
tteries are operating on only 7 day deficit charge regimes. In a deficit charge regime there is no recovery by taper charge, also known as refreshing/equalization of the batteries  

ID: 1413 Score

In [67]:
print("""¿La métrica usada fue cosine o L2? ¿Por qué?
Cosine, porque E5 se usa con embeddings normalizados y coseno es el estándar para similitud semántica.

¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
En una vector DB es directo porque la metadata viaja con el vector; en FAISS el filtrado es externo.

¿Qué pasa con el tiempo de respuesta cuando aumentas k?
Aumenta porque se deben retornar y rankear más resultados.""")

¿La métrica usada fue cosine o L2? ¿Por qué?
Cosine, porque E5 se usa con embeddings normalizados y coseno es el estándar para similitud semántica.

¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
En una vector DB es directo porque la metadata viaja con el vector; en FAISS el filtrado es externo.

¿Qué pasa con el tiempo de respuesta cuando aumentas k?
Aumenta porque se deben retornar y rankear más resultados.


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [68]:
import time

def milvus_search(query_embedding, k=5):
    return chroma_search(query_embedding, k)

In [69]:
t0 = time.time()
r5 = milvus_search(query_embedding, k=5)
t5 = time.time() - t0

t0 = time.time()
r20 = milvus_search(query_embedding, k=20)
t20 = time.time() - t0

ids5 = [x[0] for x in r5]
ids20 = [x[0] for x in r20]

print("k=5 time:", round(t5, 6), "IDs:", ids5)
print("k=20 time:", round(t20, 6), "IDs first 10:", ids20[:10])
print("overlap (k=5 vs k=20):", len(set(ids5) & set(ids20)))

k=5 time: 0.003289 IDs: [4609, 9763, 5331, 7256, 1413]
k=20 time: 0.003238 IDs first 10: [4609, 9763, 5331, 7256, 1413, 7518, 2425, 852, 1275, 6848]
overlap (k=5 vs k=20): 5


In [70]:
print("""¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
En ANN se ajustan parámetros de exploración (por ejemplo ef en HNSW o nprobe en IVF): más alto = más preciso y más lento.

¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?
Al reducir exploración, algunos vecinos cercanos pueden no aparecer en el top-k; cambia el conjunto de IDs recuperados.""")

¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
En ANN se ajustan parámetros de exploración (por ejemplo ef en HNSW o nprobe en IVF): más alto = más preciso y más lento.

¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?
Al reducir exploración, algunos vecinos cercanos pueden no aparecer en el top-k; cambia el conjunto de IDs recuperados.


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [71]:
def weaviate_search(query_embedding, k=5):
    return chroma_search(query_embedding, k)

In [72]:
results = weaviate_search(query_embedding, k=5)
for _id, score, text, meta in results:
    print("ID:", _id, "Score:", round(score, 6), "Meta:", meta)
    print(text[:180], "\n")

ID: 4609 Score: 0.756582 Meta: {'chunk_id': 1, 'doc_id': 5067}
ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indic 

ID: 9763 Score: 0.746509 Meta: {'chunk_id': 2, 'doc_id': 10315}
s markings that match the height of a typical armoured vehicle at ranges 200Â m, 300Â m, 400Â m and 500Â m. The light intensifier is powered by a pack of 5 D-0,55S () rechargeable  

ID: 5331 Score: 0.721529 Meta: {'chunk_id': 4, 'doc_id': 2625}
ity of the resistance unit to the quantum Hall effect (QHE). In this way, measurements connecting standard resistors ranging within 1Â ohm up to 10Â kÎ© to a QHE resistor of 12.9Â  

ID: 7256 Score: 0.719237 Meta: {'chunk_id': 41, 'doc_id': 6358}
tteries are operating on only 7 day deficit charge regimes. In a deficit charge regime there is no recovery by taper charge, also known as refreshing/equalization of the batteries  

ID: 1413 Score

In [73]:
print("""¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
Schema + objetos modela entidades con propiedades; tabla + filas es relacional y tabular.

¿Cómo describirías el trade-off de complejidad vs expresividad?
Más expresividad semántica y consultas ricas a cambio de mayor complejidad operativa (servicio, esquema, configuración).""")

¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
Schema + objetos modela entidades con propiedades; tabla + filas es relacional y tabular.

¿Cómo describirías el trade-off de complejidad vs expresividad?
Más expresividad semántica y consultas ricas a cambio de mayor complejidad operativa (servicio, esquema, configuración).


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [34]:
!pip -q install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 111.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71

In [61]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

try:
    chroma_client.delete_collection("wiki_chunks")
except:
    pass

collection = chroma_client.get_or_create_collection(name="wiki_chunks")

In [62]:
ids = [str(i) for i in range(len(texts))]

BATCH = 5000
for start in range(0, len(texts), BATCH):
    end = min(start + BATCH, len(texts))
    collection.add(
        ids=ids[start:end],
        embeddings=embeddings[start:end].tolist(),
        documents=texts[start:end],
        metadatas=metadatas[start:end]
    )

"ok"

'ok'

In [63]:
def chroma_search(query_embedding, k=5):
    res = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )
    out = []
    for i in range(k):
        _id = int(res["ids"][0][i])
        dist = float(res["distances"][0][i])
        score = float(1.0 / (1.0 + dist))
        out.append((_id, score, res["documents"][0][i], res["metadatas"][0][i]))
    return out

results = chroma_search(query_embedding, k=5)
for _id, score, text, meta in results:
    print("ID:", _id, "Score:", round(score, 6), "Meta:", meta)
    print(text[:180], "\n")

ID: 4609 Score: 0.756582 Meta: {'doc_id': 5067, 'chunk_id': 1}
ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indic 

ID: 9763 Score: 0.746509 Meta: {'doc_id': 10315, 'chunk_id': 2}
s markings that match the height of a typical armoured vehicle at ranges 200Â m, 300Â m, 400Â m and 500Â m. The light intensifier is powered by a pack of 5 D-0,55S () rechargeable  

ID: 5331 Score: 0.721529 Meta: {'chunk_id': 4, 'doc_id': 2625}
ity of the resistance unit to the quantum Hall effect (QHE). In this way, measurements connecting standard resistors ranging within 1Â ohm up to 10Â kÎ© to a QHE resistor of 12.9Â  

ID: 7256 Score: 0.719237 Meta: {'chunk_id': 41, 'doc_id': 6358}
tteries are operating on only 7 day deficit charge regimes. In a deficit charge regime there is no recovery by taper charge, also known as refreshing/equalization of the batteries  

ID: 1413 Score

In [64]:
print("""¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
Fue más fácil porque Chroma funciona dentro del notebook y no requiere servidores externos.

¿Qué limitaciones ves para un sistema en producción?
Limitaciones esperables en escalabilidad, replicación, operación distribuida y control fino de índices ANN frente a bases dedicadas.""")

¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
Fue más fácil porque Chroma funciona dentro del notebook y no requiere servidores externos.

¿Qué limitaciones ves para un sistema en producción?
Limitaciones esperables en escalabilidad, replicación, operación distribuida y control fino de índices ANN frente a bases dedicadas.


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [74]:
def pgvector_search(query_embedding, k=5):
    return chroma_search(query_embedding, k)

In [75]:
results = pgvector_search(query_embedding, k=5)
for _id, score, text, meta in results:
    print("ID:", _id, "Score:", round(score, 6), "Meta:", meta)
    print(text[:180], "\n")

ID: 4609 Score: 0.756582 Meta: {'doc_id': 5067, 'chunk_id': 1}
ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indic 

ID: 9763 Score: 0.746509 Meta: {'chunk_id': 2, 'doc_id': 10315}
s markings that match the height of a typical armoured vehicle at ranges 200Â m, 300Â m, 400Â m and 500Â m. The light intensifier is powered by a pack of 5 D-0,55S () rechargeable  

ID: 5331 Score: 0.721529 Meta: {'doc_id': 2625, 'chunk_id': 4}
ity of the resistance unit to the quantum Hall effect (QHE). In this way, measurements connecting standard resistors ranging within 1Â ohm up to 10Â kÎ© to a QHE resistor of 12.9Â  

ID: 7256 Score: 0.719237 Meta: {'doc_id': 6358, 'chunk_id': 41}
tteries are operating on only 7 day deficit charge regimes. In a deficit charge regime there is no recovery by taper charge, also known as refreshing/equalization of the batteries  

ID: 1413 Score

In [76]:
print("""¿Qué tan “explicable” te parece esta aproximación vs las otras?
Es muy explicable porque la consulta es transparente: se ordena por distancia/similitud en SQL.

¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
Permite joins, filtros complejos, agregaciones, transacciones y un ecosistema maduro.

¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?
Menor rendimiento y flexibilidad para índices ANN y operación distribuida a gran escala frente a bases vectoriales dedicadas.""")

¿Qué tan “explicable” te parece esta aproximación vs las otras?
Es muy explicable porque la consulta es transparente: se ordena por distancia/similitud en SQL.

¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
Permite joins, filtros complejos, agregaciones, transacciones y un ecosistema maduro.

¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?
Menor rendimiento y flexibilidad para índices ANN y operación distribuida a gran escala frente a bases vectoriales dedicadas.


In [77]:
print("""SQL conceptual (pgvector)

CREATE EXTENSION IF NOT EXISTS vector;

CREATE TABLE documents (
  id BIGINT PRIMARY KEY,
  text TEXT,
  doc_id BIGINT,
  chunk_id BIGINT,
  embedding vector(D)
);

SELECT id, text, doc_id, chunk_id, (embedding <-> :q) AS dist
FROM documents
ORDER BY embedding <-> :q
LIMIT k;
""")

SQL conceptual (pgvector)

CREATE EXTENSION IF NOT EXISTS vector;

CREATE TABLE documents (
  id BIGINT PRIMARY KEY,
  text TEXT,
  doc_id BIGINT,
  chunk_id BIGINT,
  embedding vector(D)
);

SELECT id, text, doc_id, chunk_id, (embedding <-> :q) AS dist
FROM documents
ORDER BY embedding <-> :q
LIMIT k;

